In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
from tqdm.notebook import tqdm

In [ ]:
data_dir = os.path.join("/Users/srirammandalika/Downloads/unlabeled_data/Loop3")
train_dir = os.path.join(data_dir, "/Users/srirammandalika/Downloads/unlabeled_data/Loop3/Original") 
val_dir = os.path.join(data_dir, "/Users/srirammandalika/Downloads/unlabeled_data/Loop3/val")
train_fns = os.listdir(train_dir)
val_fns = os.listdir(val_dir)
print(len(train_fns), len(val_fns))

sample_image_fp = os.path.join(train_dir, train_fns[0])
sample_image = Image.open(sample_image_fp).convert("RGB")

num_items = 2000
color_array = np.random.choice(range(256), 3*num_items).reshape(-1, 3)
print(color_array.shape)
print(color_array[:5, :])

In [ ]:
num_classes = 10
label_model = KMeans(n_clusters=num_classes)
label_model.fit(color_array)

In [ ]:
class UNet(nn.Module):
    
    def __init__(self, num_classes):
        
        super(UNet, self).__init__()
        self.num_classes = num_classes
        self.contracting_11 = self.conv_block(in_channels=3, out_channels=64)
        self.contracting_12 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_21 = self.conv_block(in_channels=64, out_channels=128)
        self.contracting_22 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_31 = self.conv_block(in_channels=128, out_channels=256)
        self.contracting_32 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_41 = self.conv_block(in_channels=256, out_channels=512)
        self.contracting_42 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.middle = self.conv_block(in_channels=512, out_channels=1024)
        self.expansive_11 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_12 = self.conv_block(in_channels=1024, out_channels=512)
        self.expansive_21 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_22 = self.conv_block(in_channels=512, out_channels=256)
        self.expansive_31 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_32 = self.conv_block(in_channels=256, out_channels=128)
        self.expansive_41 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_42 = self.conv_block(in_channels=128, out_channels=64)
        self.output = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=3, stride=1, padding=1)
        
    def conv_block(self, in_channels, out_channels):
        block = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels),
                                    nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels))
        return block
    
    def forward(self, X):
        
        contracting_11_out = self.contracting_11(X) # [-1, 64, 256, 256]
        contracting_12_out = self.contracting_12(contracting_11_out) # [-1, 64, 128, 128]
        contracting_21_out = self.contracting_21(contracting_12_out) # [-1, 128, 128, 128]
        contracting_22_out = self.contracting_22(contracting_21_out) # [-1, 128, 64, 64]
        contracting_31_out = self.contracting_31(contracting_22_out) # [-1, 256, 64, 64]
        contracting_32_out = self.contracting_32(contracting_31_out) # [-1, 256, 32, 32]
        contracting_41_out = self.contracting_41(contracting_32_out) # [-1, 512, 32, 32]
        contracting_42_out = self.contracting_42(contracting_41_out) # [-1, 512, 16, 16]
        middle_out = self.middle(contracting_42_out) # [-1, 1024, 16, 16]
        expansive_11_out = self.expansive_11(middle_out) # [-1, 512, 32, 32]
        expansive_12_out = self.expansive_12(torch.cat((expansive_11_out, contracting_41_out), dim=1)) # [-1, 1024, 32, 32] -> [-1, 512, 32, 32]
        expansive_21_out = self.expansive_21(expansive_12_out) # [-1, 256, 64, 64]
        expansive_22_out = self.expansive_22(torch.cat((expansive_21_out, contracting_31_out), dim=1)) # [-1, 512, 64, 64] -> [-1, 256, 64, 64]
        expansive_31_out = self.expansive_31(expansive_22_out) # [-1, 128, 128, 128]
        expansive_32_out = self.expansive_32(torch.cat((expansive_31_out, contracting_21_out), dim=1)) # [-1, 256, 128, 128] -> [-1, 128, 128, 128]
        expansive_41_out = self.expansive_41(expansive_32_out) # [-1, 64, 256, 256]
        expansive_42_out = self.expansive_42(torch.cat((expansive_41_out, contracting_11_out), dim=1)) # [-1, 128, 256, 256] -> [-1, 64, 256, 256]
        output_out = self.output(expansive_42_out) # [-1, num_classes, 256, 256]
        return output_out

    
    
class CityscapeDataset(Dataset):

    
    
    def __init__(self, image_dir, label_model):
        self.image_dir = image_dir
        self.image_fns = os.listdir(image_dir)
        self.label_model = label_model
        
    def __len__(self):
        return len(self.image_fns)
    
    def __getitem__(self, index):
        image_fn = self.image_fns[index]
        image_fp = os.path.join(self.image_dir, image_fn)
        image = Image.open(image_fp).convert('RGB')
        image = np.array(image)
        cityscape, label = self.split_image(image)
        label_class = self.label_model.predict(label.reshape(-1, 3)).reshape(256, 256)
        cityscape = self.transform(cityscape)
        label_class = torch.Tensor(label_class).long()
        return cityscape, label_class
    
    def split_image(self, image):
        image = np.array(image)
        cityscape, label = image[:, :256, :], image[:, 256:, :]
        return cityscape, label
    
    def transform(self, image):
        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)

In [ ]:
import torch


model_path = "/Users/srirammandalika/Desktop/U-Net-v11.pth"
device = torch.device("cpu")

model_ = UNet(num_classes=num_classes).to(device)
model_.load_state_dict(torch.load(model_path, map_location=device))


In [ ]:
test_batch_size = 16
dataset = CityscapeDataset(val_dir, label_model)
data_loader = DataLoader(dataset, batch_size=test_batch_size)

X, Y = next(iter(data_loader))
X, Y = X.to(device), Y.to(device)
Y_pred = model_(X)
print(Y_pred.shape)
Y_pred = torch.argmax(Y_pred, dim=1)
print(Y_pred.shape)

In [ ]:
inverse_transform = transforms.Compose([
    transforms.Normalize((-0.485/0.229, -0.456/0.224, -0.406/0.225), (1/0.229, 1/0.224, 1/0.225))
])

In [ ]:
import torch
import torch.nn.functional as F

def calculate_entropy(outputs, targets):
    # Convert the outputs to probabilities by applying a softmax function
    outputs = torch.softmax(outputs, dim=1)
    
    # Extract the probabilities for the correct class from the output
    correct_class_probs = outputs.gather(1, targets.unsqueeze(1))
    
    # Calculate the entropy for each pixel
    entropy = -1 * correct_class_probs * torch.log(correct_class_probs + 1e-30)
    
    return entropy

class EntropyLoss(nn.Module):
    def _init_(self):
        super(EntropyLoss, self).init()
    def forward(self, pred, target):
    # convert target to one-hot encoding
        target_onehot = F.one_hot(target, num_classes=self.num_classes).float()
    # calculate the negative of element-wise product of predicted probability and log of predicted probability
        entropy = -1 * (target_onehot * torch.log(pred+1e-30)).sum(dim=1).mean()
        return entropy

In [ ]:
criterion = nn.CrossEntropyLoss()
entropy_loss_fn = EntropyLoss()

In [ ]:
device = torch.device("cpu")

model = UNet(num_classes).to(device)
model.load_state_dict(torch.load("/Users/srirammandalika/Desktop/U-Net-v11.pth", map_location=device))
model.eval()

In [ ]:
output = model(X.cpu())
entropy = calculate_entropy(output,Y_pred.cpu())

In [ ]:
print(entropy[0].shape)
e = entropy.cpu().detach().numpy()
print(e.shape)
# print(e.shape)
for i in range(16):
    plt.imshow(e[i][0])
    plt.show()

In [ ]:
e_grid = torchvision.utils.make_grid(entropy[0])
plt.imshow(e_grid[0])

In [ ]:
fig, axes = plt.subplots(test_batch_size, 3, figsize=(3*5, test_batch_size*5))

for i in range(test_batch_size):
    
    landscape3 = inverse_transform(X[i]).permute(1, 2, 0).cpu().detach().numpy()
    label_class3 = Y[i].cpu().detach().numpy()
    label_class_predicted3 = Y_pred[i].cpu().detach().numpy()
    
    axes[i, 0].imshow(landscape3)
    axes[i, 0].set_title("Landscape")
    axes[i, 1].imshow(label_class3)
    axes[i, 1].set_title("Label Class")
    axes[i, 2].imshow(label_class_predicted3)
    axes[i, 2].set_title("Label Class - Predicted")

In [ ]:
from scipy.stats import entropy

def patchwise_entropy(image, patch_size=8):
    h, w = image.shape
    entropy_vals = []
    for i in range(0, h - patch_size + 1, patch_size):
        for j in range(0, w - patch_size + 1, patch_size):
            patch = image[i:i + patch_size, j:j + patch_size]
            hist = np.histogram(patch, bins=256, range=(0, 255))[0]
            hist = hist / np.sum(hist)
            patch_entropy = entropy(hist, base=2)
            entropy_vals.append(patch_entropy)
    return entropy_vals

# Use the Y_pred[0] as it is, assuming it's already a numpy array
image = Y_pred[0]

# Calculate patchwise entropy
entropy_vals = patchwise_entropy(image)

# Create a heatmap of the entropy values
entropy_map = np.zeros(image.shape)
k = 0
for i in range(0, image.shape[0] - 8 + 1, 8):
    for j in range(0, image.shape[1] - 8 + 1, 8):
        entropy_map[i:i + 8, j:j + 8] = entropy_vals[k]
        k += 1

# Display the entropy map
plt.imshow(entropy_map, cmap='jet')
plt.show()


In [ ]:
class CityscapeDataset(Dataset):
    def __init__(self, image_dir, label_model, num_images=1500):
        self.image_dir = image_dir
        self.label_model = label_model
        
        # Check if directory exists
        if not os.path.exists(image_dir):
            raise ValueError(f"Directory does not exist: {image_dir}")
        
        # Load only the first 'num_images' images
        self.image_fns = [f for f in sorted(os.listdir(image_dir)) if f.endswith('.jpg')]
        if not self.image_fns:
            raise ValueError(f"No JPG files found in directory: {image_dir}")
        
        self.image_fns = self.image_fns[:num_images]
        
    def __len__(self):
        return len(self.image_fns)
    
    def __getitem__(self, index):
        image_fn = self.image_fns[index]
        image_fp = os.path.join(self.image_dir, image_fn)
        image = Image.open(image_fp).convert('RGB')
        image = np.array(image)
        cityscape, label = self.split_image(image)
        
        # Ensure label is not empty
        if label.size == 0:
            raise ValueError(f"Label image is empty for {image_fn}")
        
        label_class = self.label_model.predict(label.reshape(-1, 3)).reshape(cityscape.shape[0], cityscape.shape[1])
        cityscape = self.transform(cityscape)
        label_class = torch.Tensor(label_class).long()
        return cityscape, label_class
    
    def split_image(self, image):
        cityscape = image[:, :256, :]
        label = image[:, 256:, :]
        return cityscape, label
    
    def transform(self, image):
        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)


## MiDas Depth Analyser (PAE Module)

In [ ]:
import torch
import cv2
import os
from torchvision.transforms import Compose
import matplotlib.pyplot as plt

# Load MiDaS model
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

# Choose the right model type
model_type = "DPT_Large"  # Replace with "DPT_Hybrid" or "MiDaS_small" as needed

# Select the appropriate transforms
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas.to(device)
midas.eval()

# Path to your image
filename = '/Users/srirammandalika/Downloads/unlabeled_data/Loop1/raw/0.png'

# Check if the file exists
if not os.path.isfile(filename):
    print(f"The file {filename} does not exist.")
else:
    # Read and process the image
    img = cv2.imread(filename)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Apply transforms
        input_batch = transform(img).to(device)

        # Perform depth estimation
        with torch.no_grad():
            prediction = midas(input_batch)

            # Resize to original image resolution
            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=img.shape[:2],
                mode="bilinear",
                align_corners=False,
            ).squeeze()

        # Convert to numpy array and normalize
        output = prediction.cpu().numpy()
        output_normalized = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX)

        # Apply colormap for visualization
        depth_colormap = cv2.applyColorMap(output_normalized.astype('uint8'), cv2.COLORMAP_JET)

        # Display depth map using matplotlib
        plt.imshow(depth_colormap)
        plt.title('Depth Map')
        plt.show()
    else:
        print(f"Unable to load image {filename}.")


In [ ]:
import torchvision.models as models

# Example: Load a pre-trained model (like ResNet)
model = models.resnet50(pretrained=True)
model = model.to(device)
model.eval()


In [ ]:
import torch
import torch.nn.functional as F

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.feature_maps = None
        self.register_hooks()

    def register_hooks(self):
        def forward_hook(module, input, output):
            self.feature_maps = output

        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0]

        layer = self.model._modules.get(self.target_layer)
        layer.register_forward_hook(forward_hook)
        layer.register_backward_hook(backward_hook)

    def generate_heatmap(self, input_img):
        # Forward pass
        output = self.model(input_img)

        # You need to specify the index of the class in the output
        # Here, we are taking the class with the highest output value
        score = output[:, output.argmax()]

        # Backward pass
        self.model.zero_grad()
        score.backward()

        # Weighted combination of feature maps
        weighted_combination = torch.mean(self.gradients, [2, 3], keepdim=True)
        weighted_feature_maps = F.relu((weighted_combination * self.feature_maps).sum(1))

        # Normalize the heatmap
        heatmap = weighted_feature_maps.squeeze().cpu().detach().numpy()
        heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())

        return heatmap

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Load the raw image
image_path = '/Users/srirammandalika/Downloads/unlabeled_data/Loop1/raw/0.png'  # Replace with your local path
raw_image = Image.open(image_path).convert('RGB')

# Define the transformation for the image
transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transformation to the image
input_tensor = transform(raw_image).unsqueeze(0)  # Add batch dimension

# Load the pre-trained model
model = models.resnet50(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Choose the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_tensor = input_tensor.to(device)

# Initialize Grad-CAM (Assuming GradCAM class and generate_heatmap method are defined)
grad_cam = GradCAM(model, 'layer4')

# Generate the heatmap from the model
heatmap = grad_cam.generate_heatmap(input_tensor)

# Normalize and convert heatmap to colormap for visualization
heatmap_min, heatmap_max = heatmap.min(), heatmap.max()
heatmap = np.uint8(255 * (heatmap - heatmap_min) / (heatmap_max - heatmap_min))
colored_heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Resize the heatmap to the size of the raw image
colored_heatmap = cv2.resize(colored_heatmap, (raw_image.width, raw_image.height), interpolation=cv2.INTER_AREA)

# Load MiDaS model for depth estimation
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
midas.to(device)
midas.eval()

# Prepare the image for depth estimation
# The transforms for the MiDaS model are designed to be applied to NumPy arrays, not PIL Images
midas_transform = midas_transforms.dpt_transform

# Convert the PIL image to a NumPy array
img_np = np.array(raw_image)

# Apply the MiDaS transforms
input_batch = midas_transform(img_np).to(device)

# Perform depth estimation
with torch.no_grad():
    prediction = midas(input_batch)

    # Resize to original image resolution
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=(raw_image.height, raw_image.width),
        mode="bilinear",
        align_corners=False,
    ).squeeze()

# Convert to numpy array and normalize
depth_map = prediction.cpu().numpy()
depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)

# Apply colormap for visualization
depth_colormap = cv2.applyColorMap(depth_map_normalized.astype('uint8'), cv2.COLORMAP_JET)

# Overlay the heatmap on the depth map
superimposed_img = cv2.addWeighted(depth_colormap, 0.6, colored_heatmap, 0.4, 0)

# Visualize the result
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Load your depth map and raw image here
# depth_map = cv2.imread('path_to_depth_map.png', cv2.IMREAD_UNCHANGED)
raw_image = Image.open('/Users/srirammandalika/Downloads/unlabeled_data/Loop1/raw/0.png').convert('RGB')

# Normalize the depth map to have values between 0 and 1
depth_map_normalized = cv2.normalize(depth_map, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_32F)

# Convert raw_image to numpy array if it's a PIL Image
raw_image_np = np.array(raw_image)

# Ensure the raw_image is normalized between 0 and 1
raw_image_np = raw_image_np.astype(np.float32) / 255.0

# Expand the depth map to three channels by replicating the single channel across the RGB channels
depth_map_expanded = np.repeat(depth_map_normalized[:, :, np.newaxis], 3, axis=2)

# Perform the pathwise product
pathwise_product = np.multiply(depth_map_expanded, raw_image_np)

# Display the result
plt.imshow(pathwise_product)
plt.axis('off')  # Hide the axis
plt.show()


In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Assume pathwise_product is the output from the previous code block and is available here

# Convert pathwise_product to PIL Image to apply transformation
pathwise_product_pil = Image.fromarray((pathwise_product * 255).astype('uint8'), 'RGB')

# Define the transformation for the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transformation to the pathwise product image
input_tensor = transform(pathwise_product_pil).unsqueeze(0)  # Add batch dimension

# Load the pre-trained model
model = models.resnet50(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Choose the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_tensor = input_tensor.to(device)

# Initialize Grad-CAM (Make sure to define GradCAM class or import it if it's from an external library)
grad_cam = GradCAM(model, 'layer4')

# Generate the heatmap from the model
heatmap = grad_cam.generate_heatmap(input_tensor)

# Normalize and convert heatmap to colormap for visualization
heatmap_min, heatmap_max = heatmap.min(), heatmap.max()
heatmap = np.uint8(255 * (heatmap - heatmap_min) / (heatmap_max - heatmap_min))
colored_heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Invert the colors of the heatmap
colored_heatmap = colored_heatmap[:, :, ::-1]

# Resize the heatmap to the size of the pathwise product image
colored_heatmap = cv2.resize(colored_heatmap, (pathwise_product_pil.width, pathwise_product_pil.height), interpolation=cv2.INTER_AREA)

# Overlay the heatmap on the pathwise product image
superimposed_img = cv2.addWeighted(np.array(pathwise_product_pil), 0.6, colored_heatmap, 0.4, 0)

# Visualize the result
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from scipy.stats import entropy

# Assuming GradCAM class and other necessary imports are already defined

# Your entropy and Grad-CAM code here

# Assuming Y_pred[0] is the image for which you want to create the overlay
image = Y_pred[0]  # Replace this with your actual image
entropy_vals = patchwise_entropy(image)  # Calculate entropy

# Create an entropy map
entropy_map = np.zeros(image.shape)
k = 0
for i in range(0, image.shape[0] - 8 + 1, 8):
    for j in range(0, image.shape[1] - 8 + 1, 8):
        entropy_map[i:i + 8, j:j + 8] = entropy_vals[k]
        k += 1

# Convert entropy map to PIL Image for processing
entropy_map_pil = Image.fromarray(np.uint8(entropy_map / np.max(entropy_map) * 255), 'L')
entropy_map_pil = entropy_map_pil.resize((224, 224))  # Resize to match Grad-CAM size

# Prepare the entropy map for overlay
entropy_map_pil = transforms.functional.to_tensor(entropy_map_pil).unsqueeze(0)

# Generate Grad-CAM heatmap
# ... Grad-CAM code ...

# Resize Grad-CAM heatmap to match the entropy map size
colored_heatmap = cv2.resize(colored_heatmap, (entropy_map_pil.shape[2], entropy_map_pil.shape[3]), interpolation=cv2.INTER_AREA)

# Convert Grad-CAM heatmap to tensor for overlay
colored_heatmap_tensor = transforms.functional.to_tensor(colored_heatmap).unsqueeze(0)

# Overlay the entropy map and Grad-CAM heatmap
superimposed_img = 0.6 * entropy_map_pil + 0.4 * colored_heatmap_tensor
superimposed_img = transforms.functional.to_pil_image(superimposed_img.squeeze(0))

# Visualize the result
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()


In [ ]:
class CityscapeDataset(Dataset):
    def __init__(self, image_dir, label_model, num_images=1200):
        self.image_dir = image_dir
        self.label_model = label_model

        # Check if directory exists
        if not os.path.exists(image_dir):
            raise ValueError(f"Directory does not exist: {image_dir}")

        # Load only the first 'num_images' images
        self.image_fns = [f for f in sorted(os.listdir(image_dir)) if f.endswith('.jpg')]
        if not self.image_fns:
            raise ValueError(f"No PNG files found in directory: {image_dir}")

        self.image_fns = self.image_fns[:num_images]
        
    def __len__(self):
        return len(self.image_fns)
    
    def __getitem__(self, index):
        image_fn = self.image_fns[index]
        image_fp = os.path.join(self.image_dir, image_fn)
        image = Image.open(image_fp).convert('RGB')
        image = np.array(image)
        cityscape, label = self.split_image(image)
        label_class = self.label_model.predict(label.reshape(-1, 3)).reshape(256, 256)
        cityscape = self.transform(cityscape)
        label_class = torch.Tensor(label_class).long()
        return cityscape, label_class
    
    def split_image(self, image):
        image = np.array(image)
        cityscape, label = image[:, :256, :], image[:, 256:, :]
        return cityscape, label
    
    def transform(self, image):
        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)

## DICE

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import entropy

class RawAndLabelDataset(Dataset):
    def __init__(self, raw_dir, label_dir, transform=None):
        self.raw_dir = raw_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(raw_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        raw_path = os.path.join(self.raw_dir, self.image_files[idx])
        label_filename = self.image_files[idx].replace('.jpg', '_label.jpg')  # Adjust if label filenames differ
        label_path = os.path.join(self.label_dir, label_filename)

        raw_image = read_image(raw_path).float() / 255.0
        label_image = read_image(label_path).float() / 255.0

        if self.transform:
            raw_image = self.transform(raw_image)
            label_image = self.transform(label_image)

        return raw_image, label_image

def calculate_patchwise_entropy(image, patch_size=8):
    # Assuming image is a 2D numpy array (segmentation result)
    entropy_vals = []
    for i in range(0, image.shape[0] - patch_size + 1, patch_size):
        for j in range(0, image.shape[1] - patch_size + 1, patch_size):
            patch = image[i:i + patch_size, j:j + patch_size]
            hist, _ = np.histogram(patch, bins=range(256), density=True)
            ent = entropy(hist, base=2)
            entropy_vals.append(ent)
    entropy_map = np.array(entropy_vals).reshape((image.shape[0] // patch_size, image.shape[1] // patch_size))
    return entropy_map

# Load the U-Net model
model_path = '/Users/srirammandalika/Desktop/U-Net-v11.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(num_classes=num_classes).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# Set up dataset and data loader
raw_dir = '/Users/srirammandalika/Downloads/unlabeled_data/Loop2/unsupervised-rep'
label_dir = '/Users/srirammandalika/Downloads/unlabeled_data/Loop2/US_label'
transform = transforms.Compose([transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = RawAndLabelDataset(raw_dir, label_dir, transform=transform)
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Visualize the images and entropy maps
for raw_image, label_image in loader:
    raw_image = raw_image.to(device)
    label_image = label_image.to(device)

    with torch.no_grad():
        output = model(raw_image)
        predicted = torch.argmax(output, dim=1).cpu().numpy()[0]  # Assuming batch size is 1

    entropy_map = calculate_patchwise_entropy(predicted)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(to_pil_image(raw_image.cpu().squeeze()))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(predicted, cmap='viridis')
    plt.title('Predicted Segmentation')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(entropy_map, cmap='jet')
    plt.title('Entropy Map')
    plt.axis('off')

    plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import entropy
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image
import os


# Load the U-Net model
# ... [rest of your model loading code] ...

# Set up dataset and data loader
# ... [rest of your dataset setup] ...

# Calculating entropy
entropy_values = []

for raw_image, label_image in loader:
    raw_image = raw_image.to(device)
    label_image = label_image.to(device)

    with torch.no_grad():
        output = model(raw_image)
        predicted = torch.argmax(output, dim=1).cpu().numpy()[0]  # Assuming batch size is 1

    entropy_map = calculate_patchwise_entropy(predicted)
    average_entropy = np.mean(entropy_map)
    entropy_values.append(average_entropy)

# Plotting the entropy graph
plt.figure(figsize=(10, 6))
plt.plot(range(len(entropy_values)), entropy_values, '-o')
plt.title('Average Entropy per Image')
plt.xlabel('Image Index')
plt.ylabel('Average Entropy')
plt.show()


In [ ]:
for i, (raw_image, label_image) in enumerate(loader):
    raw_image = raw_image.to(device)
    label_image = label_image.to(device)

    with torch.no_grad():
        output = model(raw_image)
        predicted = torch.argmax(output, dim=1).cpu().numpy()[0]  # Assuming batch size is 1

    entropy_map = calculate_patchwise_entropy(predicted)
    average_entropy = np.mean(entropy_map)
    entropy_values.append(average_entropy)

    print(f"Image {i}: Average Entropy = {average_entropy:.4f}")


In [ ]:
total_entropy = 0

for raw_image, label_image in loader:
    raw_image = raw_image.to(device)
    label_image = label_image.to(device)

    with torch.no_grad():
        output = model(raw_image)
        predicted = torch.argmax(output, dim=1).cpu().numpy()[0]  # Assuming batch size is 1

    entropy_map = calculate_patchwise_entropy(predicted)
    average_entropy = np.mean(entropy_map)
    entropy_values.append(average_entropy)
    total_entropy += average_entropy

combined_average_entropy = total_entropy / len(loader)
print(f"Combined Average Entropy = {combined_average_entropy:.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, axes = plt.subplots(test_batch_size, 4, figsize=(4*5, test_batch_size*5))

for i in range(test_batch_size):
    # Assuming you have the images in the correct format
    landscape = inverse_transform(X[i]).permute(1, 2, 0).cpu().detach().numpy()
    label_class = Y[i].cpu().detach().numpy()
    label_class_predicted = Y_pred[i]  # Directly use Y_pred[i] as it's already a numpy array

    # Calculate entropy and create an entropy map for the current segmented image
    entropy_vals = patchwise_entropy(label_class_predicted)
    entropy_map = np.zeros(label_class_predicted.shape)
    k = 0
    for x in range(0, label_class_predicted.shape[0] - 8 + 1, 8):
        for y in range(0, label_class_predicted.shape[1] - 8 + 1, 8):
            entropy_map[x:x + 8, y:y + 8] = entropy_vals[k]
            k += 1

    # Define high entropy threshold
    high_entropy_threshold = np.mean(entropy_vals) + np.std(entropy_vals)

    # Create a mask for high entropy patches
    high_entropy_mask = entropy_map > high_entropy_threshold

    # Replace high entropy patches in label_class_predicted with corresponding patches from label_class
    modified_segmented_image = np.copy(label_class_predicted)
    for x in range(0, label_class_predicted.shape[0] - 8 + 1, 8):
        for y in range(0, label_class_predicted.shape[1] - 8 + 1, 8):
            if high_entropy_mask[x:x + 8, y:y + 8].any():  # Check if any part of the patch is high entropy
                modified_segmented_image[x:x + 8, y:y + 8] = label_class[x:x + 8, y:y + 8]

    # Plotting
    axes[i, 0].imshow(landscape)
    axes[i, 0].set_title("Landscape")
    axes[i, 1].imshow(label_class)
    axes[i, 1].set_title("Label Class")
    axes[i, 2].imshow(label_class_predicted)
    axes[i, 2].set_title("Label Class - Predicted")
    axes[i, 3].imshow(modified_segmented_image)
    axes[i, 3].set_title("Modified Segmented Image")

plt.tight_layout()
plt.show()
